In [13]:
import sys
import os
from sklearn.decomposition import PCA
import h5py
import numpy as np
sys.path.append("..")  # Adds higher directory to python modules path.
from img_to_vec import Img2Vec
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder


In [14]:
input_path = '/home/mikaelnb/Project/notebooks/YouTubeFacesDB/data'

img2vec = Img2Vec(model='resnet-152')

print(os.listdir(input_path))

['frame_images_DB', 'sources', 'headpose_DB', 'meta_data', 'aligned_images_DB', 'images.txt', 'descriptors_DB']


/home/mikaelnb/.local/lib/python3.6/site-packages/torchvision/transforms/transforms.py:188: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


In [15]:
aligned_folder = '/aligned_images_DB/'
new_input_path = input_path + aligned_folder
unique_labels = sorted(os.listdir(new_input_path), key=lambda s: s.lower())[:2]
label_id = {}
for i, label in enumerate(unique_labels):
    label_id[i] = label
print(unique_labels, label_id)

['Aaron_Eckhart', 'Aaron_Guiel'] {0: 'Aaron_Eckhart', 1: 'Aaron_Guiel'}


In [16]:
for i in label_id:
    print(label_id[i])

Aaron_Eckhart
Aaron_Guiel


In [17]:
print(len(label_id))
pics = [None]*len(label_id)
for index in label_id:
    pics[int(index)]=[]
print(pics)


2
[[], []]


In [18]:
labels = []
features = []
vec = []

prev_index=-1

for index in label_id:    
    print(index)
    for root, dirs, files in os.walk(new_input_path + label_id[index]):
        print(prev_index == index)
        if prev_index == index:
            continue
        
        for i, name in enumerate(files):
            if i < 5 and name.endswith((".jpg")):
                img = Image.open(os.path.join(os.fsdecode(root), name))
                #print(os.path.join(os.fsdecode(root), name))
                vec = img2vec.get_vec(img)

                labels.append(label_id[index])
                features.append(vec)
            elif i >= 5:
                continue
            print(label_id[index], i+1)
        if dirs == []:
            prev_index = index

0
False
False
Aaron_Eckhart 1
Aaron_Eckhart 2
Aaron_Eckhart 3
Aaron_Eckhart 4
Aaron_Eckhart 5
True
True
1
False
False
Aaron_Guiel 1
Aaron_Guiel 2
Aaron_Guiel 3
Aaron_Guiel 4
Aaron_Guiel 5


In [21]:
#(al parecer transpose ya realiza vstack)
#features=np.vstack(features)
features=np.transpose(features)

print('labels:',len(labels))
print(features.shape)
print(features[0].shape)


labels: 10
(2048, 10)
(10,)


In [68]:
print('labels:',len(labels))
print(features.shape)

labels: 50
(2048, 50)


In [69]:
with h5py.File("data.hdf5", "w") as f:
    dset = f.create_dataset("dataset_1", data=np.array(features))

In [70]:
le = LabelEncoder()
target = le.fit_transform(labels)
with h5py.File("labels.hdf5", "w") as f:
    dset = f.create_dataset("dataset_1", data=np.array(target))